At first webdriver is imported to be able to extract data from the correct URL.

In [109]:
from selenium import webdriver
import pandas as pd
import statistics
import csv
#pip install xslxwriter
import xlsxwriter
driver = webdriver.Chrome() #,chrome_options=chrome_options)

In [83]:
driver.get('https://www.vivino.com/explore?e=eJzLLbI1VMvNzLM1UMtNrLA1NTBQS6609fNRS7Z1DQ1SKwDKpqfZliUWZaaWJOao5Rel2KrlJ1XaqpWXRMcCJYuApLGFKQAadhYz')

the URL leads to the ranking page of Vivino.com where the filters Region = Bourgogne, Type of wine =  Red wine and the full price range and all average ratings are selected. 

In [90]:
from bs4 import BeautifulSoup
res = driver.page_source.encode('utf-8')
soup = BeautifulSoup(res, "html.parser")

Below the total amount of wines left with the filter we implemented is extraced from the website. This is done by finding the right class and then split the total line of text obtained by spaces and then select the first element, since that is the amount of wines that are going to be scraped. (This total amount of wines that are going to be scraped will later on be implemented in the loop, to make sure all wines will end up in the output list)

In [91]:
total_wines = soup.find_all(attrs={"class": "layout__inner--3JC-x"})[0].text
total_wines = total_wines.split(' ')[0]
total_wines = int(total_wines)
print(total_wines)

1924


Now it is time to perform the loop to scrape all the wines. The outcome of this loop is 4 lists that include wine names, wine prices, number of reviews and the average ratings of all wines from Bourgogne region. 

First the lists are defined. Adding elements to a non-existing list is impossible.
The number of iterations is total_wines/5. The total number of wines is 5 because we chose the scroll range to be 1000. With a scroll range of 1000, 5 wines will be scraped in 1 run. We thus need to use total_wines/5 runs to scrape all wines on the website. 

The function then returns 4 lists which can be transformed in variables to perform further analysis on. 

In [112]:
def scroll_page():
    from time import sleep
        
    scroll_range = 0
    the_range = int(20)

    for _ in range(the_range):
        res = driver.page_source.encode('utf-8')
        soup = BeautifulSoup(res, "html.parser")
        
        # total number of wines in current view
        num_wines_view = len(soup.find_all(attrs={"data-testid" : "wineCard"}))
        all_wines = soup.find_all(attrs={"data-testid" : "wineCard"})
        
        # add wine attributes name, price, number of reviews and average ratings to list
        for counter in range(num_wines_view):
            
            the_name_id = soup.find_all(attrs={"data-testid": "wineCard"})[counter].find_all(attrs={"class": "wineInfoVintage__vintage--VvWlU wineInfoVintage__truncate--3QAtw"})[0].text
            the_price_id = soup.find_all(attrs={"data-testid": "wineCard"})[counter].find_all(True, {"class": ["addToCartButton__price--qJdh4" , "addToCart__subText--1pvFt addToCart__ppcPrice--ydrd5"]})[0].text
            the_reviews_id =  soup.find_all(attrs={"data-testid": "wineCard"})[counter].find_all(attrs={"class": "vivinoRating__caption--3tZeS"})[0].text
            the_rating_id = soup.find_all(attrs={"data-testid": "wineCard"})[counter].find_all(attrs={"class": "vivinoRating__averageValue--3Navj"})[0].text
            
            with open('output_odcm.csv', mode='a', newline='') as csv_file:
                csv_writer = csv.writer(csv_file)
                csv_writer.writerow([the_name_id, the_price_id, the_reviews_id, the_rating_id])
    
        # scroll down the page
        scroll_range += 2000
        driver.execute_script('window.scrollTo(0, ' + str(scroll_range) + ')')
        
        
        #update total number of wines in current view, since extra wines are added
        num_wines_view = len(soup.find_all(attrs={"data-testid" : "wineCard"}))
    
        # pause for 5 seconds
        sleep(1)
    

Now the function  scroll_page() will be runned and transformed in 4 output variables: Bourgogne_wine_names, Bourgogne_wine_prices, Bourgogne_number_of_reviews and Bourgogne_average_rating. These 4 variables are all a list of strings. 

In [113]:
scroll_page()

WebDriverException: Message: chrome not reachable
  (Session info: chrome=93.0.4577.82)


For a checkup, we check the lengths of all lists, to be sure we scraped all wines from the page. 

In [42]:
print(len(Bourgogne_wine_names))
print(len(Bourgogne_wine_prices))
print(len(Bourgogne_number_of_reviews))
print(len(Bourgogne_average_rating))

0
0
0
0


In [14]:
print(Bourgogne_number_of_reviews)

['50 beoordelingen', '411 beoordelingen', '320 beoordelingen', '318 beoordelingen', '298 beoordelingen', '266 beoordelingen', '245 beoordelingen', '238 beoordelingen', '232 beoordelingen', '215 beoordelingen', '193 beoordelingen', '177 beoordelingen', '153 beoordelingen', '137 beoordelingen', '137 beoordelingen', '133 beoordelingen', '101 beoordelingen', '100 beoordelingen', '87 beoordelingen', '57 beoordelingen', '57 beoordelingen', '53 beoordelingen', '53 beoordelingen', '52 beoordelingen', '51 beoordelingen', '50 beoordelingen', '47 beoordelingen', '44 beoordelingen', '43 beoordelingen', '42 beoordelingen', '39 beoordelingen', '36 beoordelingen', '30 beoordelingen', '29 beoordelingen', '26 beoordelingen', '396 beoordelingen', '391 beoordelingen', '338 beoordelingen', '314 beoordelingen', '243 beoordelingen', '239 beoordelingen', '236 beoordelingen', '232 beoordelingen', '222 beoordelingen', '220 beoordelingen', '216 beoordelingen', '216 beoordelingen', '214 beoordelingen', '205 beoo

The Bourgogne_price_list contains different types of strings, some say "€price" and  some say "online verkrijgbaar vanaf €prijs". To be able to perform anaylsis on this list as a whole, we need to transform the list to be of the same "layout". Afterwards we transform the elements of type "str" in the prices list into float variables. We do the same for the average_rating list

In [15]:
Bourgogne_correct_price_list = []
for ele in Bourgogne_wine_prices: 
    splitted_prices = (ele.split(" ")[-1])
    Bourgogne_correct_price_list.append(splitted_prices)
print(Bourgogne_correct_price_list)

['€17280', '€5280', '€3216', '€4130.34', '€6608.54', '€6360', '€5160', '€5520', '€3216', '€3420', '€3600', '€3648', '€2668.83', '€19500', '€3540', '€3500', '€27120', '€3456', '€6900', '€3720', '€3480', '€4980', '€3060', '€634', '€13188', '€4140', '€5040', '€3900', '€5040', '€3600', '€846.84', '€6000', '€840.95', '€20280', '€4200', '€5820', '€3770.26', '€5760', '€6680', '€5640', '€3288', '€24950', '€3180', '€24960', '€6315', '€25140', '€24600', '€2202.85', '€3180', '€3995']


In [16]:
Bourgogne_correct_price_list = [s.replace("€", "") for s in Bourgogne_correct_price_list]
for i in range(0, len(Bourgogne_correct_price_list)):
    Bourgogne_correct_price_list[i] = float(Bourgogne_correct_price_list[i])
print(Bourgogne_correct_price_list)

[17280.0, 5280.0, 3216.0, 4130.34, 6608.54, 6360.0, 5160.0, 5520.0, 3216.0, 3420.0, 3600.0, 3648.0, 2668.83, 19500.0, 3540.0, 3500.0, 27120.0, 3456.0, 6900.0, 3720.0, 3480.0, 4980.0, 3060.0, 634.0, 13188.0, 4140.0, 5040.0, 3900.0, 5040.0, 3600.0, 846.84, 6000.0, 840.95, 20280.0, 4200.0, 5820.0, 3770.26, 5760.0, 6680.0, 5640.0, 3288.0, 24950.0, 3180.0, 24960.0, 6315.0, 25140.0, 24600.0, 2202.85, 3180.0, 3995.0]


In [17]:
Bourgogne_average_rating = [s.replace(",", ".") for s in Bourgogne_average_rating]
for i in range(0, len(Bourgogne_average_rating)):
    Bourgogne_average_rating[i] = float(Bourgogne_average_rating[i])
print(Bourgogne_average_rating)

[4.9, 4.8, 4.8, 4.8, 4.8, 4.8, 4.8, 4.8, 4.8, 4.8, 4.8, 4.8, 4.8, 4.8, 4.8, 4.8, 4.8, 4.8, 4.8, 4.8, 4.8, 4.8, 4.8, 4.8, 4.8, 4.8, 4.8, 4.8, 4.8, 4.8, 4.8, 4.8, 4.8, 4.8, 4.8, 4.7, 4.7, 4.7, 4.7, 4.7, 4.7, 4.7, 4.7, 4.7, 4.7, 4.7, 4.7, 4.7, 4.7, 4.7]


In [18]:
average_Bourgogne_price = statistics.mean(Bourgogne_correct_price_list)
average_Bourgogne_rating = statistics.mean(Bourgogne_average_rating)
print(average_Bourgogne_price)
print(average_Bourgogne_rating)

7331.0922
4.772
